In [7]:
import pandas as pd
import numpy as np
from travel_distance_map import VBBAPICached, Query, GPSPoint, Position
from datetime import datetime, timedelta

# Compute travel time for an equally spaced rectangular area

In [8]:
with open('ACCESS_ID.txt') as f:
    ACCESS_ID = f.read().strip()

In [9]:
api = VBBAPICached(ACCESS_ID)

In [12]:
topleft = GPSPoint(52.522, 13.4322)
bottomright = GPSPoint(52.5005, 13.4727)
num_points = 2
dest = Position(52.5219216, 13.411026, 900100003, 'Alexanderplatz')
WALK_VELOCITY = 2.0/3.6
dt = datetime(2020, 6, 10, 5, 0, 0)

Option 1: compute travel time for a grid of points

In [25]:
df = pd.DataFrame(columns=['latitude', 'longitude', 'closest stop ID', 'closest stop name'])

lat_arr = np.linspace(topleft.lat, bottomright.lat, num_points)
lon_arr = np.linspace(topleft.lon, bottomright.lon, num_points)

for lat in lat_arr:
    for lon in lon_arr:
        origin = GPSPoint(lat, lon)
        # closest stop
        closest_stop = api.get_closest_stop(origin)
        
        # shortest travel time
        if closest_stop is None:
            # there is no stop near the origin
            sorted_trips = []
        else:
            distance_to_closest_stop = origin.distance_utf(closest_stop.point)

            trips = api.get_all_trips(closest_stop, dest, dt)
            trips_adjusted = []
            for travel_time, trip in trips:
                trips_adjusted.append((travel_time + timedelta(seconds=distance_to_closest_stop/WALK_VELOCITY), trip))

            sorted_trips = sorted(trips_adjusted, key=lambda x:x[0])
            
        if sorted_trips:
            shortest_trip = sorted_trips[0]
            shortest_trip_time = shortest_trip[0].total_seconds()
            success = True
        else:
            shortest_trip = (origin.distance_utf(dest.point)/WALK_VELOCITY, ['No trip found'])
            shortest_trip_time = origin.distance_utf(dest.point)/WALK_VELOCITY
            success = False
        shortest_trip_itinerary = shortest_trip[1]
        
        df = df.append({'latitude': lat, 
                        'longitude': lon, 
                        'closest stop latitude': closest_stop.point.lat, 
                        'closest stop longitude': closest_stop.point.lon, 
                        'closest stop ID': closest_stop.id, 
                        'closest stop name': closest_stop.name,
                        'shortest travel time': shortest_trip_time,
                        'distance to closest stop': distance_to_closest_stop,
                        'shrotest trip itinerary': shortest_trip_itinerary
                       }, ignore_index=True)

In [26]:
df

,latitude,longitude,closest stop ID,closest stop name,closest stop latitude,closest stop longitude,distance to closest stop,shortest travel time,shrotest trip itinerary
0,52.5220,13.4322,900120019,Platz der Vereinten Nationen (Berlin),52.523252,13.433057,150.807694,631.453849,"[Platz der Vereinten Nationen (Berlin), U Alex..."
1,52.5220,13.4727,900160511,Storkower Str./Paul-Junius-Str. (Berlin),52.523000,13.472448,112.494424,1042.489963,"[Storkower Str./Paul-Junius-Str. (Berlin), S S..."
2,52.5005,13.4322,900014106,Lausitzer Platz (Berlin),52.499467,13.431916,116.461989,809.631580,"[Lausitzer Platz (Berlin), U Schlesisches Tor ..."
3,52.5005,13.4727,900160008,S Rummelsburg/Hauptstr. EV (Berlin),52.500941,13.476349,251.821674,1173.279014,"[S Rummelsburg/Hauptstr. EV (Berlin), S Rummel..."


Option 2: compute lsit of stops and then compute travel time for them

In [32]:
df = pd.DataFrame(columns=['latitude', 'longitude'])

lat_arr = np.linspace(topleft.lat, bottomright.lat, num_points)
lon_arr = np.linspace(topleft.lon, bottomright.lon, num_points)
stops = []
for lat in lat_arr:
    for lon in lon_arr:
        origin = GPSPoint(lat, lon)
        # closest stop
        closest_stop = api.get_closest_stop(origin)
        stops.append(closest_stop)
        
for stop in stops:
    print(stop)
    trips = api.get_all_trips(stop, dest, dt)
    sorted_trips = sorted(trips, key=lambda x:x[0])

    if sorted_trips:
        shortest_trip = sorted_trips[0]
        shortest_trip_time = shortest_trip[0].total_seconds()
        success = True
    else:
        shortest_trip = (origin.distance_utf(dest.point)/WALK_VELOCITY, ['No trip found'])
        shortest_trip_time = origin.distance_utf(dest.point)/WALK_VELOCITY
        success = False
    shortest_trip_itinerary = shortest_trip[1]
        
    df = df.append({'latitude': stop.point.lat, 
                    'longitude': stop.point.lon, 
                    'ID': stop.id, 
                    'name': stop.name,
                    'shortest travel time': shortest_trip_time,
                    'shrotest trip itinerary': shortest_trip_itinerary
                   }, ignore_index=True)

{'id': '900120019', 'name': 'Platz der Vereinten Nationen (Berlin)', 'point': '(52.523252,13.433057)'}
{'id': '900160511', 'name': 'Storkower Str./Paul-Junius-Str. (Berlin)', 'point': '(52.523,13.472448)'}
{'id': '900014106', 'name': 'Lausitzer Platz (Berlin)', 'point': '(52.499467,13.431916)'}
{'id': '900160008', 'name': 'S Rummelsburg/Hauptstr. EV (Berlin)', 'point': '(52.500941,13.476349)'}


In [33]:
df

,latitude,longitude,ID,name,shortest travel time,shrotest trip itinerary
0,52.523252,13.433057,900120019,Platz der Vereinten Nationen (Berlin),360.0,"[Platz der Vereinten Nationen (Berlin), U Alex..."
1,52.523000,13.472448,900160511,Storkower Str./Paul-Junius-Str. (Berlin),840.0,"[Storkower Str./Paul-Junius-Str. (Berlin), S S..."
2,52.499467,13.431916,900014106,Lausitzer Platz (Berlin),600.0,"[Lausitzer Platz (Berlin), U Schlesisches Tor ..."
3,52.500941,13.476349,900160008,S Rummelsburg/Hauptstr. EV (Berlin),720.0,"[S Rummelsburg/Hauptstr. EV (Berlin), S Rummel..."


Render results

In [20]:
import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [34]:
m = folium.Map(((topleft.lat + bottomright.lat)/2.0, (topleft.lon + bottomright.lon)/2.0), zoom_start=14)
for index, row in df.iterrows():
    folium.CircleMarker((row['latitude'], row['longitude']), fill_color='black', opacity=0.4, radius=2, color='black', stroke=True, tooltip='{:.2f}'.format(row['shortest travel time'])).add_to(m)
    # folium.CircleMarker((row['closest stop latitude'], row['closest stop longitude']), fill_color='green', opacity=0.7, radius=3, color='green', stroke=True, tooltip='{:.2f}'.format(row['shortest travel time'])).add_to(m)
m